In [1]:
import json
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing.data import QuantileTransformer
from datetime import datetime
from folium import plugins
import folium
with open('../datasets/region.json') as f:
     regions = json.load(f)

In [64]:
predictions = pd.read_csv('https://raw.githubusercontent.com/jinnyto/touristcast/0d6ed676539e3995ea5bdbd7db688a49c13924a9/Hotel_stays_forecast_2018-2019.csv',parse_dates=True,index_col=0)
predictions.index.name = 'date'

,nights_IDF,nights_CVL,nights_BFC,nights_NOR,nights_HDF,nights_GES,nights_PDL,nights_BRE,nights_NAQ,nights_OCC,nights_ARA,nights_PAC,nights_COR
date,,,,,,,,,,,,,
2010-01-01,4357.10,253.94,343.08,341.09,473.95,641.67,333.33,273.63,602.06,630.13,1658.40,956.88,30.95
2010-02-01,4171.42,278.21,436.45,381.80,531.67,724.44,343.87,323.05,723.95,772.10,2020.37,1114.93,40.01
2010-03-01,5341.48,379.48,477.09,492.98,639.51,862.07,446.19,433.01,916.84,884.48,2059.83,1327.84,50.37
2010-04-01,5478.19,464.01,567.65,669.45,735.97,1007.38,500.07,580.99,1218.11,1351.76,1650.53,1727.31,185.57
2010-05-01,6203.97,582.88,674.73,755.69,800.57,1179.30,548.92,675.95,1352.00,1642.23,1530.91,2066.52,399.97
2010-06-01,6156.55,615.70,702.76,775.14,823.33,1192.85,639.38,712.65,1424.36,1721.77,1764.10,2295.96,492.30
2010-07-01,6416.40,713.93,838.74,903.39,843.23,1255.97,673.78,812.94,1847.53,2189.64,2282.64,2956.64,537.56
2010-08-01,5730.38,715.48,816.21,988.16,785.18,1317.23,736.27,1043.84,2178.51,2563.59,2278.03,3235.44,630.21
2010-09-01,5991.00,579.93,714.88,765.62,801.58,1253.54,594.56,724.66,1509.44,1801.24,1702.37,2256.28,487.23


In [66]:
ll = pd.read_csv('../datasets/tc.csv')
ll.sort_values(by=['Region'],inplace=True)
dn = pd.read_csv('../datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
predictions.index.name = 'date'
df = pd.concat([dn, predictions],sort=True)
start_date='2015-01-01'
end_date='2019-12-01'
#years = df.loc[start_date:end_date,dn.columns.str.startswith('nights')]
df
start_date='2015-01-01'
end_date='2017-12-01'
df =df.loc[df.index >='2015-01-01']
df

,nights_ARA,nights_BFC,nights_BRE,nights_COR,nights_CVL,nights_GES,nights_HDF,nights_IDF,nights_NAQ,nights_NOR,nights_OCC,nights_PAC,nights_PDL
date,,,,,,,,,,,,,
2015-01-01,1886.960000,360.920000,339.420000,27.820000,273.580000,710.760000,534.690000,4518.600000,664.040000,351.790000,662.610000,949.710000,353.500000
2015-02-01,2265.220000,444.690000,390.470000,29.650000,306.430000,793.120000,561.510000,4288.110000,771.300000,404.070000,760.790000,1101.710000,366.470000
2015-03-01,2215.680000,472.680000,501.190000,42.810000,363.040000,893.440000,656.680000,5084.780000,953.220000,509.040000,876.240000,1330.280000,445.150000
2015-04-01,1674.930000,550.540000,599.450000,142.000000,459.730000,1056.910000,777.410000,5693.810000,1273.000000,658.970000,1263.150000,1807.220000,554.290000
2015-05-01,1652.700000,685.500000,729.430000,384.650000,561.180000,1280.620000,835.620000,6136.030000,1431.830000,774.680000,1615.400000,2217.800000,622.070000
2015-06-01,1908.600000,721.690000,755.530000,460.830000,582.990000,1254.610000,845.570000,6311.220000,1520.430000,788.180000,1585.300000,2433.310000,705.550000
2015-07-01,2400.890000,838.700000,897.850000,517.320000,700.650000,1371.800000,877.510000,6775.390000,1926.650000,892.190000,2037.070000,2988.150000,755.740000
2015-08-01,2559.620000,850.960000,1085.330000,644.390000,722.130000,1444.560000,837.480000,6348.850000,2327.860000,955.000000,2383.620000,3363.680000,848.250000
2015-09-01,1817.690000,715.530000,803.580000,506.620000,598.690000,1310.560000,861.970000,6020.960000,1591.180000,739.410000,1736.380000,2356.550000,651.780000


In [82]:
dn = pd.read_csv('../datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
df = pd.concat([dn, predictions], axis=0, sort=True)
regions = [x.replace('nights_','') for x in list(df.columns)]
regions.sort()
start_date='2015-01-01'
end_date='2019-12-01'
years = df.loc[df.index >='2015-01-01']
years = years.T


In [83]:
#min_max_scaler = MinMaxScaler(feature_range=(0.1, 1.0))
quantile_transformer =  QuantileTransformer(output_distribution='uniform')
years = pd.DataFrame(quantile_transformer.fit_transform(years), columns=years.columns, index=years.index) 
years.sort_index(inplace=True)
years.index = ll.Region
years.reset_index()
years = years.T
years.columns = [str(col) + '_M' for col in years.columns]
for cols in years.columns:
    region = cols.replace('_M', '')
    years[region +'_La'] = ll.loc[ll['Region'] == region, 'Latitude'].iloc[0]
    years[region +'_Lo'] = ll.loc[ll['Region'] == region, 'Longitude'].iloc[0]
years = years.reindex(sorted(years.columns), axis=1)
region_geo = os.path.join('.', 'region.json')
time_index = years.index.to_series().apply(lambda x: datetime.strftime(x, '%Y-%m-%d')).tolist()
years_data = (np.array(years).reshape(-1,3))

In [84]:
heatmap_data=[]
for idx,year in enumerate(years.index):
    df = pd.DataFrame(years_data[idx*13:idx*13+13],index=regions,columns=['Latitude','Longitude','Magnitude'])
    one_year_data=np.array(df)    
    heatmap_data.append(one_year_data.tolist())

In [85]:
from jinja2 import Template
m = folium.Map(location=[46.9, 2.], zoom_start=6)
time_index = years.index.to_series().apply(lambda x: datetime.strftime(x, '%b %Y')).tolist()
hm = plugins.HeatMapWithTime(
    heatmap_data,
    index=time_index,
    auto_play=True,
    radius=40
)
hm.add_to(m)
geo_json_data = json.load(open(region_geo))
g = folium.GeoJson(geo_json_data,style_function=lambda x: {'fillColor':'#ffff00','color':'#6d6d5b','weight':2,'opacity':0.6}).add_to(m)
g._template = Template("""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = {};

                {{this.get_name()}}.style = function(feature) {return feature.properties.style;};

                {{this.get_name()}}.highlightStyle = function(feature) {return {
                        weight: 3,
                        color: '#6d6d5b',
                        dashArray: '',
                        fillOpacity: 0.4
                        };
                    };

                {{this.get_name()}}.onEachFeature = function onEachFeature(feature, layer) {
                    layer.on({
                        mouseover: function(e) {
                            e.target.setStyle({{this.get_name()}}.highlightStyle(e.target.feature));},
                        mouseout: function(e) {
                            {{this.get_name()}}.geoJson.resetStyle(e.target);},
                        click: function(e) {
                            {{this._parent.get_name()}}.fitBounds(e.target.getBounds());}
                        });
                    };

                {{this.get_name()}}.geoJson = L.geoJson(
                    {% if this.embed %}{{this.style_data()}}{% else %}"{{this.data}}"{% endif %},{
                        style : {{this.get_name()}}.style,
                        onEachFeature: {{this.get_name()}}.onEachFeature
                        })
                    .addTo({{this._parent.get_name()}});
            {% endmacro %}
""")

m.add_child(g)
m.save('2015_2019_heatmapwithtime.html')
m
